# Quick Review of cluster architecture

(diagram goes here)


# Quick Review of Slurm allocations

$ srun [options] batch.sh

option | example | comment
-------|--------|---
-c _cores_ | -c 20 | cores/task on single node
-n _tasks_ | -n 4 | mpi progs only
-N _nodes_ | -N 10 | never useful
-t _time_  | -t 7-, -t 3:00 | job killed if exceeded
--mem=_mem_ | --mem=16g | ditto
--mem-per-cpu=_mem_ | --mem-per-cpu=8g | ditto
-J _name_ | -J myjob | for user only


# Thinking about memory requirements
- default 5GB/core on our clusters
- strictly enforced; jobs exceeding limit killed
- you can request custom memory per node or core


# Determining memory requirements
After successful run, determine actual usage:
- ssh to node, run 
    "top -o RES -u _netid_"
    look at RES column
- /usr/bin/time -a _prog args_
- sacct -o 'JobID,MaxRSS,MaxVMSize' -j _jobid_
- remora

# Remora
https://github.com/TACC/remora

module load REMORA
remora prog args ...

This will create a directory: remora_jobid

Copy (rsync) to local computer, open remora_summary.html with browser 

